# Simple TFIDF clustering

In [1]:
file = open("./questions.txt","r+")

o_dataset = file.readlines()

In [10]:
import collections
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.cluster import KMeans
from pprint import pprint
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob

In [2]:
def word_tokenizer(text):
            #tokenizes and stems the text
            #print(text)
            tokens = word_tokenize(text.lower())
            stemmer = PorterStemmer()
            #print("1",tokens)
            tokens = [stemmer.stem(t) for t in tokens if t not in stopwords.words('english')]
            #print("2",tokens)
            return tokens


def cluster_sentences_tfidf(sentences, nb_of_clusters=5):
        tfidf_vectorizer = TfidfVectorizer(tokenizer=word_tokenizer,
                                        max_df=0.9,
                                        min_df=0.1,
                                        lowercase=True)
        d=[]
        
        #builds a tf-idf matrix for the sentences
        tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
        
        f=tfidf_vectorizer.get_feature_names()
        #print(tfidf_vectorizer.get_feature_names)
        kmeans = KMeans(n_clusters=nb_of_clusters)
        kmeans.fit(tfidf_matrix)
        for doc in range(len(sentences)):
            feature_index = tfidf_matrix[doc,:].nonzero()[1]
            #print(feature_index)
            tfidf_scores = zip(feature_index, [tfidf_matrix[doc, x] for x in feature_index])
            d.append(dict(tfidf_scores))
        clusters = collections.defaultdict(list)
        for i, label in enumerate(kmeans.labels_):
                clusters[label].append(i)
        return dict(clusters),d

if __name__ == "__main__":
    #print("Hello")
    #dataset = ['Name a few important dates during the World WAr 2',' Write a short note on cvivl war',' what are subatomic particles?','does dust particles cause itching as the eyes?','Explain seven years itch as a movie']
    b,mm = cluster_sentences_tfidf(o_dataset,5)
    #print(b)
    print(mm[3])
    """for i in b:
        print("__________________________________CLUSTER "+str(i)+"______________________________________")"""
    """for j in b[i]:
            print(o_dataset[j])
        print("\n\n")"""

{2: 0.6813139060940797, 3: 0.7319913670002042}


# Clustering using Word2Vec Gensim

In [238]:
file = open("./paper.txt","r+")
#file = open("./questions.txt","r+")
o_dataset = file.readlines()

In [239]:
#o_dataset.append("Mention the major similarities between RNA and DNA")

In [240]:
from gensim.models import word2vec
from nltk.cluster import *

In [4]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('/home/meghana/PES/SemVI/NLP/GoogleNews-vectors-negative300.bin', binary=True)  

In [241]:
import math

def tf(w,sen):
    b=0.3
    return math.log(1+sen.count(w))
    #return b+((1-b)*sen.count(w)/max(list(Counter(sen.split()).values())))
    #return sen.count(w)/len(sen.split())
def idf(w,c):
    s = sum([int(w in i) for i in c])
    return math.log(len(c)/s)
def tfidf(c):
    final=[]
    for i in c:
        #print(i)
        res = {}
        for j in i.split():
            #print(j,tf(j,i))
            res[j] = tf(j,i)*idf(j,c)
            '''if(j.lower() == "describe" or j.lower() == "light"):
                print(j,tf(j,i),idf(j,c),res[j])'''
            #print(j,res[j])
        final.append(res)
    return final

In [242]:
def lemmatize(sentence):
    sent = TextBlob(sentence)
    tag_dict = {"J": 'a', 
                "N": 'n', 
                "V": 'v', 
                "R": 'r'}
    words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]    
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)

In [243]:
"does" in stopwords.words("english")

True

In [244]:
from string import punctuation as punc
from nltk.corpus import stopwords
import re
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() 

def preprocess(s,model):
    r = "["+punc+"]"
    regex = re.compile(r)
    r1 = "[\n]"
    regex1 = re.compile(r1)
    sw = stopwords.words("english")
    sw.append("that's")
    sw.append("us")
    review = s
    review1 = review
    review1 = list(map(lambda x: re.sub(regex1," ",x),review))
    review1 = list(map(lambda x: x.strip(),review))
    #review = list(map(lambda x: x.lower(),review))
    review = [" ".join(list(filter(lambda x: x.lower() not in sw,i.split()))) for i in review if i not in sw]
    review = list(map(lambda x: re.sub(regex1," ",x),review))
    review = list(map(lambda x: x.strip(),review))
    
    review = [lemmatize(i) for i in review]
    print(review1[2])
    print(review[2])

    review = list(map(lambda x: re.sub(regex,"",x),review))
    '''for i in review:
        review = list(filter(lambda x: x in model.vocab,i))'''
    return (review,review1)

In [245]:
'''import random
random.shuffle(o_dataset)

o_dataset'''
"Mention the major similarities between RNA and DNA" in o_dataset

False

In [246]:
dataset,o_da = preprocess(o_dataset,model)

Note the similarity and dissimilarity between intensive subsistence farming and commercial farming practiced in India.
Note similarity dissimilarity intensive subsistence farm commercial farming practice India


In [247]:
"Mention the major similarities between RNA and DNA" in o_da

False

In [248]:
dataset[4],o_da[4]

('Explain example development manufacturing industry measure economic strength country',
 'Explain with examples why the development of manufacturing industries being measured by economic strength of a country')

In [249]:
from collections import Counter

In [250]:
def get_unique(x):
    setx = set()
    return [i for i in x if not (i in setx or setx.add(i))]

In [251]:
dataset = get_unique(dataset)

In [252]:
o_da = get_unique(o_da)

In [253]:
d = tfidf(dataset)


In [254]:
print("Total number of questions -> ",len(o_dataset)," | Unique questions ->",len(dataset))

Total number of questions ->  100  | Unique questions -> 100


In [255]:
c = list(set(" ".join(dataset).split()))
len(c)

305

In [256]:
len(set(dataset)),len(dataset)

(100, 100)

In [257]:
dataset1 = dataset

In [258]:
k=0
vec = {}
new=[]
for i in c:
    if i in model.vocab:
        new.append(i)
    else:
        print(i)

creditExplain
formedExplain
favour
1857
1907
nlost
wcareerindiaco


In [259]:
import numpy as np

In [260]:
x = model[np.array(new)]

In [261]:
vec = dict(zip(new,x))

In [223]:
b=[]
cc = np.zeros((300,))
for index,sentence in enumerate(dataset):
    cc = np.zeros((300,))
    count = len(sentence.split())
    #print("count->",count)
    count = 0
    for index1,i in enumerate(sentence.split()):
        try:
            
            weight = d[index][i]
            '''print("+++++++++++++++++++++++=")'''
            count+=weight
            """if index==100:
                print(weight,i)"""
            """if index1 in mm[index]:
                if i =="Rinderpest":
                    print("__________________",i,mm[index][index1])
                    break
                weight = mm[index][index1]    
            else:

                if i == "Rinderpest":
                    print(i)
                weight=0.01"""
            #print(weight)
            cc = np.add(cc,weight*vec[i])
            print("--------------------------")
            #print("**",i)
        except Exception as e:
            '''print("yp",e)
            print("except",weight,i)'''
            if weight>1:
                count+=weight
                cc = np.add(cc,weight*np.full((300,),0.5))
                #print(weight)
        #print(i,weight)
    cc = cc/count
    #print(count,cc)
    b.append(cc)

--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
-

In [224]:
"""from sklearn.cluster import KMeans

km = KMeans(n_clusters=7,random_state=50)
a = km.fit(b)"""

'from sklearn.cluster import KMeans\n\nkm = KMeans(n_clusters=7,random_state=50)\na = km.fit(b)'

# Kmeans

In [225]:
cluster = KMeansClusterer(num_means = 10,distance = util.cosine_distance, repeats=30)
a = cluster.cluster(b,assign_clusters=True)

In [226]:
len(o_da),len(a)

(251, 251)

In [227]:
"Mention the major similarities between RNA and DNA" in b

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """Entry point for launching an IPython kernel.


False

In [228]:
prev=0
li=[]
cluster1 = KMeansClusterer(num_means = 10,distance = util.cosine_distance, repeats=30)
for i in sorted(list(zip(o_da,a)),key = lambda x: x[1]):
    if i[1]!=prev:
        print("-"*90)
    prev=i[1]
    print(i)

('Describe the structure of benzene.', 0)
('Why is benzene dangerous.', 0)
('How does the structure of carbon atoms affect the stability of benzene?', 0)
('Where is butane and propane used?', 0)
('What is coal tar?', 0)
('Why is chlorine a halogen?', 0)
('How does Benzene differ from cheese.', 0)
('Why are junk foods so oily?', 0)
('How are benzene double bonds stable?', 0)
('What is the ideal gas law?', 0)
('Are all F block elements radioactive?', 0)
('What is the half life of radioactive elements?', 0)
('What are the differences between solids, liquids and gasses?', 0)
------------------------------------------------------------------------------------------
('How do alkalis differ from halogens and d block elements?', 1)
('What are halogens?', 1)
('Why is carbon inert?', 1)
('Explain the role Sodium as a metals', 1)
('Iron has the highest ionization enthalpy. Justify.', 1)
('What is a dry cell in a battery?', 1)
('Explain cathode and anode reactions at battery terminals?', 1)
('Diff

In [229]:
prev=0
li=[]
cluster1 = KMeansClusterer(num_means = 10,distance = util.cosine_distance, repeats=30)
for i in sorted(list(zip(o_da,a)),key = lambda x: x[1]):
    if i[1]!=prev:
        print("-"*90)
    prev=i[1]
    print(i)


('Describe the structure of benzene.', 0)
('Why is benzene dangerous.', 0)
('How does the structure of carbon atoms affect the stability of benzene?', 0)
('Where is butane and propane used?', 0)
('What is coal tar?', 0)
('Why is chlorine a halogen?', 0)
('How does Benzene differ from cheese.', 0)
('Why are junk foods so oily?', 0)
('How are benzene double bonds stable?', 0)
('What is the ideal gas law?', 0)
('Are all F block elements radioactive?', 0)
('What is the half life of radioactive elements?', 0)
('What are the differences between solids, liquids and gasses?', 0)
------------------------------------------------------------------------------------------
('How do alkalis differ from halogens and d block elements?', 1)
('What are halogens?', 1)
('Why is carbon inert?', 1)
('Explain the role Sodium as a metals', 1)
('Iron has the highest ionization enthalpy. Justify.', 1)
('What is a dry cell in a battery?', 1)
('Explain cathode and anode reactions at battery terminals?', 1)
('Diff

# DBSCAN

In [262]:
from sklearn.cluster import DBSCAN


In [263]:
from collections import Counter

def get_unique(x):
    setx = set()
    return [i for i in x if not (i in setx or setx.add(i))]

def doit(o_dataset,model):
    dataset,o_da = preprocess(o_dataset,model)
    #print(dataset)
    dataset = get_unique(dataset)
    o_da = get_unique(o_da)
    #print(len(o_da))
    d = tfidf(dataset)
    #print("Total number of questions -> ",len(o_dataset)," | Unique questions ->",len(dataset))
    c = list(set(" ".join(dataset).split()))
    k=0
    vec = {}
    new=[]
    for i in c:
        if i in model.vocab:
            new.append(i)
        else:
            pass
            #print(i)
    x = model[np.array(new)]

    vec = dict(zip(new,x))

    b=[]
    cc = np.zeros((300,))
    for index,sentence in enumerate(dataset):
        cc = np.zeros((300,))
        count = len(sentence.split())
        #print("count->",count)
        count = 0
        for index1,i in enumerate(sentence.split()):
            try:

                weight = d[index][i]
                '''print("+++++++++++++++++++++++=")'''
                count+=weight
                """if index==100:
                    print(weight,i)"""
                """if index1 in mm[index]:
                    if i =="Rinderpest":
                        print("__________________",i,mm[index][index1])
                        break
                    weight = mm[index][index1]    
                else:

                    if i == "Rinderpest":
                        print(i)
                    weight=0.01"""
                #print(weight)
                cc = np.add(cc,weight*vec[i])
                #print("--------------------------")
                #print("**",i)
            except Exception as e:
                '''print("yp",e)
                print("except",weight,i)'''
                if weight>1:
                    count+=weight
                    cc = np.add(cc,weight*np.full((300,),0.5))
                    #print(weight)
            #print(i,weight)
        cc = cc/count
        #print(count,cc)
        b.append(cc)
    return b,o_da

In [264]:
dataset1 = o_dataset
num = 1
eps = 0.2;min_samples = 2
while(True):
    print("IN NEW ITER")
    b,o_da = doit(dataset1,model)
    print(len(o_da))
    dbs = DBSCAN(eps=eps,min_samples=min_samples,metric="cosine")
    x = dbs.fit(b)
    x_labels = list(set(x.labels_))
    print(x_labels)
    li = []
    prev = -1
    for i in sorted(list(zip(o_da,x.labels_)),key = lambda x: x[1]):
        if i[1]==-1:
            li.append(i[0])
            continue
        if i[1]!=prev:
            print("\n\n----New Cluster "+str(num)+"----\n\n")
            num+=1
        prev=i[1]
        print(i)
    dataset1 = li
    eps+=0.1
    if(len(x_labels)<2):
        break
print(li)

IN NEW ITER
Note the similarity and dissimilarity between intensive subsistence farming and commercial farming practiced in India.
Note similarity dissimilarity intensive subsistence farm commercial farming practice India
100
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, -1]


----New Cluster 1----


('Differentiate between intensive subsistence farming and commercial farming practiced in India.', 0)
('State the differences between commercial farming and intensive subsistence farming', 0)


----New Cluster 2----


('Why is development of manufacturing industries being measured by economic strength of a country', 1)
('Explain with examples why the development of manufacturing industries being measured by economic strength of a country', 1)


----New Cluster 3----


('Mention five functions of political parties performed in a democracy.', 2)
('What are the functions performed by political parties in a democracy?', 2)
('Describe any five major function

In [88]:
"Mention the major similarities between RNA and DNA" in b

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """Entry point for launching an IPython kernel.


False

In [89]:

print(set(x.labels_))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, -1}


In [90]:
x.labels_
li = []
for i in sorted(list(zip(o_da,x.labels_)),key = lambda x: x[1]):
    if i[1]==-1:
        li.append(i[0])
    if i[1]!=prev:
        print("-"*90)
    prev=i[1]
    print(i)

------------------------------------------------------------------------------------------
('What is light?', -1)
('Describe the phenomenon of light', -1)
('How does light enable us to see?', -1)
('What are the two theories of light?', -1)
('Describe the wave theory of light', -1)
('Describe the particle theory of light', -1)
('Explain magnetic resonance', -1)
('What are the four types of magnetism?', -1)
('What are hard magnets?', -1)
('What are soft magnets?', -1)
('Differentiate between types of magnets.', -1)
('How is sound different from light?', -1)
('What wave is a sound wave?', -1)
('Explain how amplitude varies with distance in sound waves.', -1)
('What magnetic fields are used by motors', -1)
('What magnetic fields are used by generators', -1)
("Show the Fleming's left hand rule for motors.", -1)
('When should the left and right hand rules be used respectively?', -1)
('Explain reflection of light', -1)
('Explain refraction of light', -1)
('Explain diffraction of light', -1)
(

214

In [154]:
len(li)

177

In [155]:
len(bb)


187

In [163]:
bb,o_da = doit(li,model)


Explain magnetic resonance
Explain magnetic resonance
133
Total number of questions ->  133  | Unique questions -> 133
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
--------------------------
-----------------

In [170]:
dbs = DBSCAN(eps=0.60,min_samples=2,metric="cosine")

In [171]:
len(o_da)

133

In [172]:
x = dbs.fit(bb)
print(set(x.labels_))

{0, 1, -1}


In [173]:
x.labels_

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [169]:
li = []
for i in sorted(list(zip(o_da,x.labels_)),key = lambda x: x[1]):
    if i[1]==-1:
        li.append(i[0])
    if i[1]!=prev:
        print("-"*90)
    prev=i[1]
    print(i)

------------------------------------------------------------------------------------------
('How does light enable us to see?', -1)
('Explain magnetic resonance', -1)
('Differentiate between types of magnets.', -1)
('How is sound different from light?', -1)
('What wave is a sound wave?', -1)
('Explain how amplitude varies with distance in sound waves.', -1)
('Explain reflection of light', -1)
('What angle does light bounce of a rough surface', -1)
('What is the change in sound loudness effect described by a moving object at changing distances?', -1)
('Chemical Kinetics vs surface chemistry. Explain.', -1)
('Why is carbon inert?', -1)
('What is refractive index of a surface?', -1)
('Describe the structure of benzene.', -1)
('Why is benzene dangerous.', -1)
('Who invented Penicillin?', -1)
('How did vaccines affect the world?', -1)
('How do plants absorb light?', -1)
('Explain the role Sodium as a metals', -1)
('Why are atoms of Sodium held close to each other?', -1)
('Why is the sky blu

In [136]:
bb = doit(li,model)

What is a magnetic field?
magnetic field
['light enable see', 'two theory light', 'magnetic field', 'unit magnetic field', 'Explain magnetic resonance', 'Explain term Paramagnetism Diamagnetism Antiferromagnetism Antiferrimagnetism', 'four type magnetism', 'hard magnet', 'soft magnet', 'Differentiate type magnet', 'Explain amplitude varies distance sound wave', 'Explain Doppler effect light', 'Explain Doppler effect sound', 'magnetic field use motor', 'magnetic field use generator', 'Show Fleming s left hand rule motor', 'left right hand rule use respectively', 'Explain reflection light', 'diffraction vary refreaction', 'reflection vary diffraction', 'reflection vary refraction', 'angle light bounce rough surface', 'Given surface chemistry property adsorption adsorption affect surface temperature', 'element color', 'Explain phenomenon color light', 'light seven color white light see sun', 'change sound loudness effect describe move object change distance', 'Chemical Kinetics v surface 

In [150]:
"What is light?" in li

True

In [147]:
dbs = DBSCAN(eps=0.1,min_samples=2,metric="cosine")

bb[0].shape

(300,)

In [148]:
x = dbs.fit(bb)
print(set(x.labels_))

x.labels_

{0, 1, 2, -1}


array([-1, -1, -1, -1, -1,  0, -1, -1, -1, -1, -1,  1,  1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  2,
        2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0, -1, -1, -1, -1, -1,
       -1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0, -1, -1])

In [149]:
li = []
for i in sorted(list(zip(o_da,x.labels_)),key = lambda x: x[1]):
    if i[1]==-1:
        li.append(i[0])
    if i[1]!=prev:
        print("-"*90)
    prev=i[1]
    print(i)

------------------------------------------------------------------------------------------
('What is light?', -1)
('Describe the phenomenon of light', -1)
('How does light enable us to see?', -1)
('What are the two theories of light?', -1)
('Describe the wave theory of light', -1)
('What is a magnetic field?', -1)
('What are the units of magnetic field?', -1)
('Explain magnetic resonance', -1)
('Explain the terms Paramagnetism, Diamagnetism, Antiferromagnetism, Antiferrimagnetism', -1)
('What are the four types of magnetism?', -1)
('Differentiate between types of magnets.', -1)
('How is sound different from light?', -1)
('What wave is a sound wave?', -1)
('Explain how amplitude varies with distance in sound waves.', -1)
('Explain Doppler effect of light', -1)
('Explain Doppler effect of sound', -1)
('What magnetic fields are used by motors', -1)
('What magnetic fields are used by generators', -1)
("Show the Fleming's left hand rule for motors.", -1)
('When should the left and right han

In [36]:
x.labels_
li = []
for i in sorted(list(zip(o_da,x.labels_)),key = lambda x: x[1]):
    '''if i[1]==-1:
    li.append(i[0])'''
    if i[1]!=prev:
        print("-"*90)
    prev=i[1]
    print(i)

------------------------------------------------------------------------------------------
('What is light?', -1)
('Describe the phenomenon of light', -1)
('How does light enable us to see?', -1)
('What are the two theories of light?', -1)
('Describe the wave theory of light', -1)
('Describe the particle theory of light', -1)
('Explain magnetic resonance', -1)
('What are the four types of magnetism?', -1)
('What are hard magnets?', -1)
('What are soft magnets?', -1)
('Differentiate between types of magnets.', -1)
('How is sound different from light?', -1)
('What wave is a sound wave?', -1)
('Explain how amplitude varies with distance in sound waves.', -1)
('What magnetic fields are used by motors', -1)
('What magnetic fields are used by generators', -1)
("Show the Fleming's left hand rule for motors.", -1)
('When should the left and right hand rules be used respectively?', -1)
('Explain reflection of light', -1)
('Explain refraction of light', -1)
('Explain diffraction of light', -1)
(

In [ ]:
dbs = DBSCAN(eps=0.40,min_samples=2,metric="cosine")

In [180]:
import numpy as np

def avg_sentence_vector(words, model, num_features,vc):
    #function to average all words vectors in a given paragraph
    featureVec = np.zeros((num_features,), dtype="float32")
    nwords = 0

    for word in words:
        if word in vc:
            nwords = nwords+1
            featureVec = np.add(featureVec, model[word])

    if nwords>0:
        featureVec = np.divide(featureVec, nwords)
    return featureVec

In [181]:
from sklearn.metrics.pairwise import cosine_similarity
#get average vector for sentence 1
sentence_1 = "this is sentence number one"
sentence_1_avg_vector = avg_sentence_vector(sentence_1.split(), vc=model.vocab,model=model, num_features=300)

#get average vector for sentence 2
sentence_2 = "this is sentence number two"
sentence_2_avg_vector = avg_sentence_vector(sentence_2.split(), model=model, vc=model.vocab,num_features=300)

sen1_sen2_similarity =  cosine_similarity(sentence_1_avg_vector.reshape(-1,1),sentence_2_avg_vector.reshape(-1,1))

In [11]:
from nltk import word_tokenize,sent_tokenize

def w2v(s1,s2):
    if s1==s2:
        return 1
    s1 = preprocess(s1)
    s2 = preprocess(s2)
    

In [9]:
sen1_sen2_similarity.shape

NameError: name 'sen1_sen2_similarity' is not defined

In [36]:
h = dict(sorted(list(h.items()),key=lambda x:x[0]))
for i in h:
    print("___________________________________________CLUSTER ",i+1,"______________________________________")
    for j in h[i]:
        print(o_da[j],end="\n\n")

___________________________________________CLUSTER  1 ______________________________________
Who is a disabled person Give one example each of two types of disabilities.

How do you describe a disabled person. Explain the disabilities with examples

What are the characteristics of a disabled person

Write any four facts about tsunami.

What are the characteristics of tsunami

How does tsunami occur

How is awareness among the people created by means of mass communication

Suggest two facts which should be borne in mind while selecting a site for a building in coastal areas.

What are the features that has to be considered while selecting a site for a building in coastal areas.

Describe two different approaches by which the number of the poor are estimated in the country.

How are poor estimated in the country

What are the ways of finding out number of the poor in the country

The revolt of [1857] marked a turning point in the history of India. Support this statement by giving four fa

In [35]:
h={}
for i,j in enumerate(km.labels_):
    if j not in h:
        h[j] = [i]
    else:
        h[j].append(i)

In [63]:
mm.get_feature_names()

AttributeError: get_feature_names not found